In [189]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os

In [105]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\Patrick\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [106]:
base_url = 'https://battlefieldtracker.com/bfv/leaderboards/stats/all/Wins?type=stats&page=1'

driver.get(base_url)

# Wait for page to load
time.sleep(7)

content = driver.page_source.encode('utf-8').strip()
soup = bs(content, 'html.parser')

Note that this code opened a web browser. Please do not close it, if you plan on following along with the code and executing the cells. The code below automates the browser so data can be scraped from various pages.

The HTML here is actually pretty well organized. And represents something like the image below. The difference between the image and the page generally is that the page contains 100 rows.

<img src='leaderboard_example.png'>

Each player is a row in a table on the page. I want just the body, where the rows are, i.e. I can ignore the thead.

In [107]:
player_table = soup.find("table",{'class':'trn-table'}).tbody
player_rows = player_table.find_all('tr')

Each row in the table is itself a table. The rows of each player row proceed horizontally, left to right, as opposed to vertically. They are all named decently, too. There are a number of features to keep here: rank, username, stat highlight (the stat the players are organized by, here, wins) and stat-collapse (here, rounds played.) In addition, the player's platform can be retrieved from the relative path to their profile.

In [108]:
player_data = player_rows[-1].find_all('td')
player_data

[<td class="rank" data-v-4dbc6408="" data-v-754518f9=""><span data-v-4dbc6408="" data-v-754518f9="" data-v-7a5012a4="">100</span></td>,
 <td class="username" data-v-4dbc6408="" data-v-754518f9=""><div class="text" data-v-4dbc6408="" data-v-754518f9=""><a class="" data-v-4dbc6408="" data-v-754518f9="" href="/bfv/profile/xbl/TightWolf"><div class="avatar" data-v-4dbc6408=""><img alt="TightWolf's Avatar" class="picture" data-v-4dbc6408="" loading="lazy" src="https://imgsvc.trackercdn.com/url/size(32)/https%3A%2F%2Fimages-eds-ssl-ssl.xboxlive.com%2Fimage%3Furl%3DKT_QTPJeC5ZpnbX.xahcbrZ9enA_IV9WfFEWIqHGUb5P30TpCdy9xIzUMuqZVCfbdZLL26ddnKGsETOKJmRbdWb220u73XyNom2XdDEuyB6dU77ty4zFaQxQhVulcL_LGD3VOFkaORqVdn8utqXGw3XeVJ8ksfnDVMejNXSAPro-%26format%3Dpng%26format%3Dpng%26w%3D240%26h%3D240/image.jpg"/> <!-- --></div> <svg class="platform-icon platform-icon platform-xbl" data-v-4dbc6408="" data-v-b17d31f6="" viewbox="0 0 88 88"><path d="M39.73 86.91c-6.628-.635-13.338-3.015-19.102-6.776-4.83-3.15-5.

In [109]:
player_rows[0].find_all('td')[0].text

'\n  1\n'

In [110]:
bftracker_url = 'https://www.battlefieldtracker.com'
player_profile = bftracker_url + player_data[1].a['href']
player_profile

'https://www.battlefieldtracker.com/bfv/profile/xbl/TightWolf'

In [111]:
bftracker_url = 'https://www.battlefieldtracker.com'
stat_dict = {'Rank': [], 'Username':[], 'Platform':[], 'Wins':[], 'Rounds Played':[], 'Profile':[]}

def parse_player_row(player_data, stat_dict):
    # Identify and store features
    player_rank = int(player_data[0].text.strip())
    player_username = player_data[1].find('span', {'class':'trn-ign__username'}).text.strip()
    player_platform = player_data[1].a['href'].split('/')[-2]
    player_wins = int(player_data[-2].text.strip().replace(',', ''))
    player_rounds_played =  int(player_data[-1].text.strip().replace(',', ''))
    player_profile = bftracker_url + player_data[1].a['href']
    
    # Add features to dictionary
    stat_dict['Rank'].append(player_rank)
    stat_dict['Username'].append(player_username)
    stat_dict['Platform'].append(player_platform)
    stat_dict['Wins'].append(player_wins)
    stat_dict['Rounds Played'].append(player_rounds_played)
    stat_dict['Profile'].append(player_profile)
    return stat_dict

def parse_player_rows(player_rows, stat_dict):
    for i,row in enumerate(player_rows):
        player_data = row.find_all('td')
        stat_dict = parse_player_row(player_data, stat_dict)
    return stat_dict
stat_dict = parse_player_rows(player_rows, stat_dict) 
print(stat_dict)

{'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], 'Username': ['themadbat2', 'lHluslHlKuslHl', 'jawasandcrawler', 'Z1nYoRiTa', 'Arjen10_oOorWhat', 'BoomKapoow', 'D3athD3al3r GR', 'xXhaydar_ayedXx', 'RagePooch', 'MaxiqYT', 'I Mr GoRi I', 'DASE 101', 'MELEE_OVERDRIVE', 'ChosenOne718', 'Frindly-Fire-4', 'EL-mari-ano98', 'xK1nG R3L1Cx', 'Predatory--Grin', 'Tiger766', 'UmairKingz', 'Cyborg JRS', 'TaruShiba_tv', 'Itendtokill', 'xXTrYagAiMXx', 'KRAG_Nerugui', 'XxGtOwNgOoN702xX', 'ViolentGeeks', 'noraneko-tom-3k', 'mugiwara890', 'BillyTheKid3434', 'LUKESCORPION', 'STIG_YY', 'warbonet', 'fares_8086', 'EnergisedKnight', 'DatelessBook58', 'ZigZag07',

In [112]:
profiles = stat_dict['Profile']
req = requests.get('https://battlefieldtracker.com/bfv/profile/psn/themadbat2/overview').content
soup2 = bs(req)

In [10]:
driver.get(profiles[0])

# Wait for page to load
time.sleep(5)

content = driver.page_source.encode('utf-8').strip()
ex_profile_soup = bs(content, 'html.parser') # an example profile

There are a lot of player stats on each profile page, both overall and broken down by class. The easiest way toget these is using css selectors, for the most part. Note that here, as with class specific stats, I have no interest in whether the player was in the top X% of the category. The part of the page I am processing now looks like this:

<img src='lifetime_stats.png'>

In [11]:
def get_player_lifetime_stats(profile_soup, stat_dict):
    # Collect lifetime stats for a player
    # Get lifetime hours played across all classes
    lifetime_hours = float(profile_soup.select('span.playtime')[0].text.strip().split(' ')[0][:-1].replace(',', ''))
    if 'lifetime_hours' in stat_dict:
            stat_dict['lifetime_hours'].append(lifetime_hours)
    else:
        stat_dict['lifetime_hours'] = [lifetime_hours]

    # Identify the divs with the lifetime stats
    overall_stat_table = profile_soup.select('div.main')[0]
    stat_names_values = overall_stat_table.select('div.numbers > span')
    stat_names_values[0]['class']

    # Loop to collect stats
    i = 0
    while i < len(stat_names_values):
        stat_name = stat_names_values[i].text
        stat_value = float(stat_names_values[i+1].text.replace(',', '').replace('%', '').strip())
        if stat_name in stat_dict:
            stat_dict[stat_name].append(stat_value)
        else:
            stat_dict[stat_name] = [stat_value]
        i += 2

    return stat_dict

print('Results from example profile')
get_player_lifetime_stats(ex_profile_soup, {})

Results from example profile


{'lifetime_hours': [6623.0],
 'Score/Min': [378.97],
 'K/D': [2.62],
 'Kills': [509087.0],
 'Kills/Min': [1.28],
 'Win %': [72.6],
 'Wins': [20567.0],
 'Deaths': [194413.0],
 'Assists': [104705.0],
 'Damage': [58736066.0],
 'Heals': [2291510.0],
 'Revives': [156498.0],
 'Resupplies': [169257.0]}

Next, I collect the stats for each player broken out by class, as well as by vehicle use. That is in a table like the below. The HTML organizing this information is a bit less readable, but there fortunately only two actual HTML tables on the page, and the former contains the desired data. Each row of the table consists of several table data elements. Each of those table data elements has a span with class name "name," which contains the values I want to retrieve. I define a list of stat categories I want to retrieve manually. Then for each class, I add the name of the class under consideration to the stat name, ex. medic_score.

<img src='class_stats.png'>

In [12]:
stat_categories = ['Score', 'Score/Min', 'Kills', 'Kills/Min', 'K/D']
class_stat_table = ex_profile_soup.find_all('tbody')[0].find_all('tr') # each element here is a row in the table pictured above

Below is an example row

In [13]:
class_row = class_stat_table[0].find_all('td')
class_name = class_row[0].select('span.name')[0].text # get the name of the class the row represents
class_hours = float(class_row[0].select('span.sub')[0].text[:-1].replace(',', ''))
stat_dict = {} # Reset stat dict to limit output
# add class time played to stat dictionary
if f'{class_name}_hours' in stat_dict:
        stat_dict[f'{class_name}_hours'].append(class_hours)
else:
    stat_dict[f'{class_name}_hours'] = [class_hours]

# get the stats
for i, td in enumerate(class_row[2:]):
    stat_name = f'{class_name}_{stat_categories[i]}'
    class_stat = td.select('span.name')[0].text.replace(',', '')
    if stat_name in stat_dict:
        stat_dict[stat_name].append(class_stat)
    else:
        stat_dict[stat_name] = [class_stat]

print(stat_dict)

{'Medic_hours': [3754.0], 'Medic_Score': ['89930936'], 'Medic_Score/Min': ['399.24'], 'Medic_Kills': ['335717'], 'Medic_Kills/Min': ['1.49'], 'Medic_K/D': ['2.70']}


We can see from the above that this adds the class specific stats from the row. All that is left to do for the player profile is to loop over the other classes as well. I clear the dictionary of stats to make it a bit more readable.

In [14]:
def get_player_class_stats(profile_soup, stat_dict):
    stat_categories = ['Score', 'Score/Min', 'Kills', 'Kills/Min', 'K/D']
    class_stat_table = profile_soup.find_all('tbody')[0].find_all('tr') # each element here is a row in the table pictured above
    class_row = class_stat_table[0].find_all('td')
    for row in class_stat_table:
        class_row = row.find_all('td')
        class_name = class_row[0].select('span.name')[0].text # get the name of the class the row represents
        
        # add class time played to stat dictionary
        class_hours = class_row[0].select('span.sub')[0].text
        time_split = class_hours.split(' ')
        if len(time_split) > 1:
            class_hours = time_split
            # if time played in XXh YYm format
            if 'h' in class_hours[0]:
                class_hours = float(class_hours[0][:-1]) * 60 + float(class_hours[1][:-1])
                class_hours /= 60
            # if time played in XXm YYs format
            elif 'm' in class_hours[0]:
                class_hours = float(class_hours[0][:-1]) + float(class_hours[1][:-1])/60
                class_hours /= 60
        else:
            class_hours = class_hours[:-1].replace(',', '')
        
                
        if f'{class_name}_hours' in stat_dict:
                stat_dict[f'{class_name}_hours'].append(class_hours)
        else:
            stat_dict[f'{class_name}_hours'] = [class_hours]

        # get the stats
        for i, td in enumerate(class_row[2:]):
            stat_name = f'{class_name}_{stat_categories[i]}'
            class_stat = td.select('span.name')[0].text.replace(',', '')
            if stat_name in stat_dict:
                stat_dict[stat_name].append(class_stat)
            else:
                stat_dict[stat_name] = [class_stat]
    return stat_dict

get_player_class_stats(ex_profile_soup, {})

{'Medic_hours': ['3754'],
 'Medic_Score': ['89930936'],
 'Medic_Score/Min': ['399.24'],
 'Medic_Kills': ['335717'],
 'Medic_Kills/Min': ['1.49'],
 'Medic_K/D': ['2.70'],
 'Recon_hours': ['1225'],
 'Recon_Score': ['28085034'],
 'Recon_Score/Min': ['382.10'],
 'Recon_Kills': ['92579'],
 'Recon_Kills/Min': ['1.26'],
 'Recon_K/D': ['2.55'],
 'Assault_hours': ['1023'],
 'Assault_Score': ['20594554'],
 'Assault_Score/Min': ['335.31'],
 'Assault_Kills': ['47079'],
 'Assault_Kills/Min': ['0.77'],
 'Assault_K/D': ['2.47'],
 'Support_hours': ['551'],
 'Support_Score': ['9534705'],
 'Support_Score/Min': ['288.21'],
 'Support_Kills': ['30167'],
 'Support_Kills/Min': ['0.91'],
 'Support_K/D': ['2.47'],
 'Tanker_hours': ['36'],
 'Tanker_Score': ['1774991'],
 'Tanker_Score/Min': ['813.47'],
 'Tanker_Kills': ['4388'],
 'Tanker_Kills/Min': ['2.01'],
 'Tanker_K/D': ['0.00'],
 'Pilot_hours': ['32'],
 'Pilot_Score': ['344479'],
 'Pilot_Score/Min': ['176.11'],
 'Pilot_Kills': ['634'],
 'Pilot_Kills/Min': [

The next step is to do this for all of the players on a results page. An effective way to do this limits the number of times I have to send requests to the server. First, I load a leaderboard page. From this page, I get all of the profiles for the players on the page. Then I go to each profile and collect data from the profile. Conveniently, code for each part of this task has already been written.

In [15]:
def get_leaderboard_page(leaderboard_url):
    '''
    Takes a link to a leaderboard page, loads it in the browser and returns the HTML for information loaded on the page in a format that is easy to parse
    '''
    driver.get(leaderboard_url)

    # Wait for page to load
    time.sleep(3)

    content = driver.page_source.encode('utf-8').strip()
    leaderboard_soup = bs(content, 'html.parser')
    return soup

def parse_leaderboard_page(leaderboard_url, stat_dict):
    # Load the leaderboard page
    leaderboard_soup = get_leaderboard_page(leaderboard_url)

    # Get the rows on the page, each representing basic information about a player
    player_table = soup.find("table",{'class':'trn-table'}).tbody
    player_rows = player_table.find_all('tr')

    stat_dict = parse_player_rows(player_rows, stat_dict)

    return stat_dict



### Parsing the Profiles

In [16]:
stat_dict.keys()

dict_keys(['Medic_hours', 'Medic_Score', 'Medic_Score/Min', 'Medic_Kills', 'Medic_Kills/Min', 'Medic_K/D'])

In [17]:
def get_profile_page(profile_url):
    '''
    Takes a link to a profile page, loads it in the browser and returns the HTML for information loaded on the page in a format that is easy to parse
    '''
    driver.get(profile_url)

    # Wait for page to load
    time.sleep(7)

    content = driver.page_source.encode('utf-8').strip()
    profile_soup = bs(content, 'html.parser')

    return profile_soup

def parse_profiles(stat_dict):
    '''
    For each profile on a leaderboard page, add data from the profile to the stat_dict
    '''
    profiles = stat_dict['Profile']
    for profile_url in tqdm(profiles):
        profile_soup = get_profile_page(profile_url)

        # Get the player's overall stats
        stat_dict = get_player_lifetime_stats(profile_soup, stat_dict)

        # Get the player's class stats
        stat_dict = get_player_class_stats(profile_soup, stat_dict)

        time.sleep(7)
    return stat_dict

### Parse Leaderboard Page and Associated Profiles

In [18]:
base_url = 'https://battlefieldtracker.com/bfv/leaderboards/stats/all/Wins?type=stats&page=1'
stat_dict = {'Rank': [], 'Username':[], 'Platform':[], 'Wins':[], 'Rounds Played':[], 'Profile':[]}

def parse_leaderboard_page_and_profiles(leaderboard_url, stat_dict):
    # Get basic leaderboard info
    stat_dict = parse_leaderboard_page(base_url, stat_dict)

    # get info from profiles
    stat_dict = parse_profiles(stat_dict)
    return stat_dict

parse_leaderboard_page_and_profiles(base_url, stat_dict)

 22%|██▏       | 22/100 [06:26<22:50, 17.57s/it]


IndexError: list index out of range

In [ ]:
time_played = '6h 20'
time_played = time_played.split(' ')
time_played = float(time_played[0][:-1]) * 60 + float(time_played[1])
time_played /= 60
time_played

6.333333333333333

# Profile Approach 2: Use Selenium to Load JSON
NTS: this approach seems to work much better. Further, I only need the leaderboard to load profiles

## Leaderboard

In [173]:
url = 'https://api.tracker.gg/api/v1/bfv/standard/leaderboards?type=stats&platform=all&board=WINS&skip=0&take=100'
driver.get(url)
leaderboard_content = driver.page_source.encode('utf-8').strip()
soup = bs(leaderboard_content)
leaderboard_json = json.loads(soup.body.pre.text)['data']['items']

In [178]:
ex_player = leaderboard_json[0]

sd = {}
def parse_player(player_json, stat_dict):
    player_username = player_json['id']
    player_platform = player_json['owner']['metadata']['platformSlug']
    player_id = f'{player_platform}/{player_username}'
    
    return player_id

def parse_leaderboard(leaderboard_url, stat_dict):
    # Load the URL
    driver.get(leaderboard_url)
    leaderboard_content = driver.page_source.encode('utf-8').strip()
    soup = bs(leaderboard_content)
    leaderboard_json = json.loads(soup.body.pre.text)['data']['items']
    
    # Get names from leaderboard
    for player_json in leaderboard_json:
        if 'player_id' in stat_dict:
            stat_dict['player_id'].append(parse_player(player_json, stat_dict))
        else:
            stat_dict['player_id'] = [parse_player(player_json, stat_dict)]
            
    return stat_dict

parse_leaderboard(url, {})

{'player_id': ['psn/themadbat2',
  'psn/lhluslhlkuslhl',
  'psn/jawasandcrawler',
  'origin/z1nyorita',
  'psn/arjen10_ooorwhat',
  'xbl/boomkapoow',
  'xbl/d3athd3al3r gr',
  'psn/xxhaydar_ayedxx',
  'origin/ragepooch',
  'psn/maxiqyt',
  'xbl/i mr gori i',
  'xbl/dase 101',
  'psn/melee_overdrive',
  'xbl/chosenone718',
  'origin/frindly-fire-4',
  'psn/el-mari-ano98',
  'xbl/xk1ng r3l1cx',
  'psn/predatory--grin',
  'origin/tiger766',
  'origin/umairkingz',
  'xbl/cyborg jrs',
  'psn/tarushiba_tv',
  'psn/itendtokill',
  'psn/xxtryagaimxx',
  'origin/krag_nerugui',
  'psn/xxgtowngoon702xx',
  'origin/violentgeeks',
  'psn/noraneko-tom-3k',
  'psn/mugiwara890',
  'xbl/billythekid3434',
  'psn/lukescorpion',
  'origin/stig_yy',
  'xbl/warbonet',
  'psn/fares_8086',
  'xbl/energisedknight',
  'xbl/datelessbook58',
  'psn/zigzag07',
  'psn/sishiscrage',
  'origin/douyutv_1427108',
  'xbl/alexolibaz',
  'xbl/wingy117',
  'xbl/fab50400',
  'xbl/gummy28ducks',
  'psn/ha-y-nboy16',
  'origi

## Overall History
These are summary statistics across classes.

In [155]:
sd = {}
def parse_history_for_player(history_json, stat_dict):
    for stat in history_json:
        stat_value = history_json[stat]['value']
        stat_percentile = history_json[stat]['percentile']
        
        # Add value to stat_dict
        if stat+'_value' in stat_dict:
            stat_dict[stat+'_value'].append(stat_value)
        else:
            stat_dict[stat+'_value'] = [stat_value]
            
        # Add percentile to stat_dict
        if stat+'_percentile' in stat_dict:
            stat_dict[stat+'_percentile'].append(stat_percentile)
        else:
            stat_dict[stat+'_percentile'] = [stat_percentile]
            
    return stat_dict  

parse_history_for_player(history_json, sd)

{'scorePerMinute_value': [527.0199862832441],
 'scorePerMinute_percentile': [95.5],
 'kdRatio_value': [3.7899292531766826],
 'kdRatio_percentile': [97.7],
 'kills_value': [328386],
 'kills_percentile': [99.9],
 'deaths_value': [86647],
 'deaths_percentile': [None],
 'damage_value': [35412436],
 'damage_percentile': [99.9],
 'assists_value': [47715],
 'assists_percentile': [None],
 'killsAggregated_value': [338653],
 'killsAggregated_percentile': [None],
 'assistsAsKills_value': [10267],
 'assistsAsKills_percentile': [None],
 'shotsTaken_value': [8903033],
 'shotsTaken_percentile': [None],
 'shotsHit_value': [1676939],
 'shotsHit_percentile': [None],
 'shotsAccuracy_value': [18.835592320055426],
 'shotsAccuracy_percentile': [None],
 'killStreak_value': [53],
 'killStreak_percentile': [None],
 'dogtagsTaken_value': [4145],
 'dogtagsTaken_percentile': [None],
 'avengerKills_value': [31111],
 'avengerKills_percentile': [None],
 'saviorKills_value': [41379],
 'saviorKills_percentile': [None

## Class stats

In [129]:
sd = {}
def parse_class_stats(class_json, stat_dict):
    '''
    Retrieve class-specific stats for a specific player for a single class.
    '''
    class_name = class_json['metadata']['name']
    
    # Add class name to stat_dict       
    for stat in list(class_json['stats'].keys())[1:]: # The first entry is player rank, which we don't need
        stat_percentile = class_json['stats'][stat]['percentile']
        stat_value = class_json['stats'][stat]['value']
        
        stat_name = f'{class_name}_{stat}' # ex Assault_kills
        
        # Add stat value to dictionary
        if stat_name+'_value' in stat_dict:
            stat_dict[stat_name+'_value'].append(stat_value)
        else:
            stat_dict[stat_name+'_value'] = [stat_value]
        
        # Add stat percentile to dictionary
        if stat_name+'_percentile' in stat_dict:
            stat_dict[stat_name+'_percentile'].append(stat_percentile)
        else:
            stat_dict[stat_name+'_percentile'] = [stat_percentile]
            
    return stat_dict

def parse_classes_for_player(classes_json, stat_dict):
    '''
    Retrieve class-specific stats for a specific player for all classes.
    '''
    for class_json in classes_json:
        stat_dict = parse_class_stats(class_json, stat_dict)
    
    return stat_dict

parse_classes_for_player(classes_json, sd)

{'Assault_kills_value': [38963],
 'Assault_kills_percentile': [None],
 'Assault_deaths_value': [9501],
 'Assault_deaths_percentile': [None],
 'Assault_killsPerMinute_value': [1.1582342449464922],
 'Assault_killsPerMinute_percentile': [None],
 'Assault_kdRatio_value': [4.100936743500684],
 'Assault_kdRatio_percentile': [99.1],
 'Assault_timePlayed_value': [2018430],
 'Assault_timePlayed_percentile': [None],
 'Assault_shotsFired_value': [810051],
 'Assault_shotsFired_percentile': [None],
 'Assault_shotsHit_value': [189575],
 'Assault_shotsHit_percentile': [None],
 'Assault_shotsAccuracy_value': [23.402847475035525],
 'Assault_shotsAccuracy_percentile': [None],
 'Assault_score_value': [17206444],
 'Assault_score_percentile': [99.3],
 'Assault_scorePerMinute_value': [511.48763376932226],
 'Assault_scorePerMinute_percentile': [94.0],
 'Medic_kills_value': [261305],
 'Medic_kills_percentile': [None],
 'Medic_deaths_value': [69281],
 'Medic_deaths_percentile': [None],
 'Medic_killsPerMinute_v

In [181]:
def parse_player_stats(stat_dict):
    for player_id in tqdm(stat_dict['player_id']):
        
        # Get overall history for player
        api_url = f"https://api.tracker.gg/api/v2/bfv/standard/profile/{player_id}?"

        driver.get(api_url)
        history_content = driver.page_source.encode('utf-8').strip()
        soup = bs(history_content)
        history_json = json.loads(soup.body.pre.text)['data']['segments'][0]['stats']
        
        stat_dict = parse_history_for_player(history_json, sd)
                
        # Get class info for the user in question
        api_url = f"https://api.tracker.gg/api/v2/bfv/standard/profile/{player_id}/segments/class"

        driver.get(api_url)
        class_content = driver.page_source.encode('utf-8').strip()
        soup = bs(class_content)
        classes_json = json.loads(soup.body.pre.text)['data']
        
        stat_dict = parse_classes_for_player(classes_json, stat_dict)
        
        time.sleep(1)
        
    return stat_dict

stat_dict = parse_player_stats(stat_dict)
        

In [158]:
url = 'https://api.tracker.gg/api/v1/bfv/standard/leaderboards?type=stats&platform=all&board=WINS&skip=0&take=100'
driver.get(url)
leaderboard_content = driver.page_source.encode('utf-8').strip()
soup = bs(leaderboard_content)
leaderboard_json = json.loads(soup.body.pre.text)['data']['items']

{'scorePerMinute_value': [527.0199862832441,
  378.9796463583056,
  549.2700590780313,
  252.2832295013709,
  668.2528136883133,
  316.0791284508746,
  412.962166668772,
  440.69305660700047,
  332.34250386398764,
  318.51935064045483,
  739.1470378600478,
  378.9796463583056,
  549.2700590780313,
  252.2832295013709,
  668.2528136883133,
  316.0791284508746,
  412.962166668772,
  440.69305660700047,
  332.34250386398764,
  318.51935064045483,
  739.1470378600478,
  734.823560787251,
  393.8050235879128,
  266.95111204026824,
  256.8988060719768,
  589.1971038101908,
  224.34261678231633,
  338.55828534686003,
  268.33501802427287,
  217.7509494928438,
  666.1953493685165,
  901.3902647805586,
  605.4887020670808,
  289.4714164753533,
  370.3390406800243,
  554.8170696089567,
  430.04831716818455,
  443.02036569823395,
  251.12420280447975,
  678.3736431803216,
  507.3557179838587,
  477.089768235855,
  560.6924985369633,
  390.1693037104708,
  442.97527791008025,
  635.5561018337995,


In [182]:
url = 'https://api.tracker.gg/api/v1/bfv/standard/leaderboards?type=stats&platform=all&board=WINS&skip=0&take=100'
def scrape_page(leaderboard_url):
    # Scrape the page
    stat_dict = {}
    stat_dict = parse_leaderboard(url, stat_dict)
    stat_dict = parse_player_stats(stat_dict)
    
    return stat_dict

def scrape_site(skip_init=0):
    skip = skip_init
    while skip < 78800:
        leaderboard_url = f'https://api.tracker.gg/api/v1/bfv/standard/leaderboards?type=stats&platform=all&board=WINS&skip={skip}&take=100'
        
        # scrape page
        stat_dict = scrape_page(leaderboard_url)
        
        # Load previous progess, if any
        current_iter = pd.DataFrame.from_dict(stat_dict)
        for file_ in os.listdir():
            if 'bfvstats' in file_:
                previous_iter = pd.read_csv(file_)
                temp_df = pd.concat([previous_iter, current_iter])
                
                os.remove(file_)
                
                current_iter.to_csv(f'bfvstats_skip{skip}.csv')
            else:
                current_iter.to_csv(f'bfvstats_skip{skip}.csv')
        
        skip += 100


100%|██████████| 100/100 [03:11<00:00,  1.92s/it]


In [201]:
os.listdir()

['.git',
 'a.txt',
 'bfv_players.ipynb',
 'chromedriver.exe',
 'class_stats.png',
 'leaderboard_example.png',
 'lifetime_stats.png',
 'test.csv',
 'test2.csv']

In [198]:
a = pd.DataFrame.from_dict(stat_dict)
b = pd.DataFrame.from_dict(stat_dict)


In [200]:
pd.concat([a,b])

,scorePerMinute_value,scorePerMinute_percentile,kdRatio_value,kdRatio_percentile,kills_value,kills_percentile,deaths_value,deaths_percentile,damage_value,damage_percentile,...,Tanker_shotsFired_value,Tanker_shotsFired_percentile,Tanker_shotsHit_value,Tanker_shotsHit_percentile,Tanker_shotsAccuracy_value,Tanker_shotsAccuracy_percentile,Tanker_score_value,Tanker_score_percentile,Tanker_scorePerMinute_value,Tanker_scorePerMinute_percentile
0,378.979646,71.0,2.619080,93.0,509343,99.9,194474,None,58762846,99.9,...,0,None,0,None,0.0,None,1774991,94.0,813.469753,64.0
1,549.270059,96.6,5.598978,99.0,657208,99.9,117380,None,71931790,99.9,...,0,None,0,None,0.0,None,159756,69.0,420.410526,24.0
2,252.283230,28.0,1.751625,79.0,420418,99.9,240016,None,51001813,99.9,...,0,None,0,None,0.0,None,84699,58.0,1693.980000,81.0
3,668.252814,99.0,5.202617,98.9,938318,99.9,180355,None,107543331,99.9,...,0,None,0,None,0.0,None,9943381,99.5,748.410432,61.0
4,316.079128,51.0,2.542558,93.0,442278,99.9,173950,None,50978898,99.9,...,0,None,0,None,0.0,None,35013,41.0,2059.588235,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,375.873837,70.0,3.544016,97.3,319610,99.9,90183,None,36289105,99.9,...,0,None,0,None,0.0,None,734987,88.0,644.160386,54.0
96,345.410175,61.0,1.524280,70.0,183720,99.7,120529,None,21594569,99.7,...,0,None,0,None,0.0,None,10493769,99.5,463.117040,31.0
97,392.764015,75.0,2.403558,91.0,255628,99.9,106354,None,32135411,99.9,...,0,None,0,None,0.0,None,38231016,99.9,502.266458,38.0
98,430.972072,83.0,1.180991,52.0,117857,99.1,99795,None,13320833,99.0,...,0,None,0,None,0.0,None,505344,84.0,18716.444444,98.8


# Full search
https://api.tracker.gg/api/v1/bfv/standard/leaderboards?type=stats&platform=all&board=WINS&skip=78800&take=100

max value for take is 100
skip should go from 0 to 78800, iterating by 100

In [194]:
skip = 0


print(skip)

78800


In [193]:
os.remove('test.csv')

# Find Number of Leaderboard Page
I did not know how many pages the leaderboard has. Candidly, I found this out manually through trial and error, which only took a couple of minutes. There are 791.

You could do this programmatically, but given it is polite to wait in between sending requests to servers, this would take longer.

A potential programatic approach could go as follows: each leaderboard url ends in a page number. Call this i. You could increase i in increasingly larger step sizes until you no longer load pages with HTML in the form expected. Then I could be decreased in increasingly larger step size until it loaded a page with increasingly larger step sizes until it found a page of the form desired. This could be repeated until it converged on the last page. This is only one fairly naive approach. Generally, it's a search through a sequence of integers that is unbounded on one side.